# Quadratic Programming (QP)

## What is Quadratic Programming?
A **Quadratic Program (QP)** is an optimization problem where:
- the **objective function is quadratic**
- the **constraints are linear**

In other words:
> **QP = LP + a quadratic objective**

---

## Standard QP Formulation

A common formulation (used in MATLAB, Pyomo solvers, etc.) is:

$\min \left( \frac{1}{2} x^T H x + f^T x \right)$

---

## Components Explained

### Decision Variables
$
x =
\begin{bmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{bmatrix}
$

This vector contains all decision variables of the problem.

---

### Quadratic Term (Curvature)
$
H =
\begin{bmatrix}
h_{11} & h_{12} \\
h_{21} & h_{22}
\end{bmatrix}
$

- $H$ is a **constant matrix** defining the quadratic coefficients
- It determines how strongly large values of \(x\) are penalized
- If $H$ is **positive semidefinite**, the problem is **convex** (good news for solvers)

The quadratic expression:  
$ x^T H x $  
produces squared terms like $x_1^2, x_2^2$, and cross-terms like $x_1 x_2$.

The factor $\frac{1}{2}$ is included by convention to simplify derivatives.

---

### Linear Term
$
f =
\begin{bmatrix}
f_1 \\
f_2 \\
\vdots \\
f_n
\end{bmatrix}
$

- This is the same linear term used in Linear Programming
- It shifts the minimum of the quadratic function

---

## Constraints

All constraints in QP are **linear**.

### Variable Bounds
$ LB \leq x \leq UB $

- $LB$ and $UB$ are vectors of lower and upper bounds
- Each entry corresponds to one decision variable

---

### Equality Constraints
$ A_{eq} x = b_{eq} $

- $A_{eq}$: matrix of size $n_{eq} \times n$
- $b_{eq}$: vector of size $n_{eq} \times 1$
- Each row represents one equality constraint

---

### Inequality Constraints
$ A_{ineq} x \leq b_{ineq} $

- $A_{ineq}$: matrix of size $n_{ineq} \times n$
- $b_{ineq}$: vector of size $n_{ineq} \times 1$
- Each row represents one inequality constraint

---

## Intuition

- **LP** → flat objective (straight line / plane)
- **QP** → curved objective (bowl-shaped)
- Constraints stay linear
- The quadratic term discourages extreme solutions and encourages smoothness

---
## <b style="color:red">IMPORTANT NOTE</b>
The matrix formulation shown above is used internally by many solvers (and commonly shown in MATLAB-style interfaces).

When modeling problems in **Pyomo**, constraints and objectives are written **directly as equations**, not in matrix form.  
Pyomo automatically converts these expressions into the appropriate matrix representation before passing them to the solver.

The matrix form is presented here **for explanation purposes only**, to help understand how QP solvers work under the hood.

## TL;DR
Quadratic Programming is **Linear Programming written in matrix form with a quadratic objective**:

$\min \left( \frac{1}{2} x^T H x + f^T x \right)$

Same variables.  
Same constraints.  
Just a smoother objective.

--- 
## Production Cost Optimization Problem
### Problem Description
A manufacturer has developed an empirical model of its production system and has determined that the daily production cost is given by:

$\text{cost} = 0.4x_1^2 - 5x_1 + x_2^2 - 6x_2 + 50$

where $x_1$ and $x_2$ are the decision variables representing the number of hours per day that Unit 1 and Unit 2 are operated, respectively. The objective is to determine the optimal operating hours for both units that minimize the total production cost.

The manufacturer must produce at least 20,000 units per day.  
Unit 1 is an older but durable machine that produces 2,700 units per hour and can operate for up to 18 hours per day.  
Unit 2 is a newer and faster machine that produces 3,600 units per hour but can operate for at most 9 hours per day.

Note:
* The objective function is quadratic because it has only linear and squared terms. The quadratic terms model increasing operating costs due to wear, energy consumption, and inefficiencies at higher utilization levels.

* A quadratic function can also have bilinear ($x_1x_2$) terms which products of the two independent variables.

### Mathematical Formulation
#### Objective Function
This is the objective function that we want to minimize  
$\text{cost} = 0.4x_1^2 - 5x_1 + x_2^2 - 6x_2 + 50$

#### Constraints
**1. Production target**  
$ 2700 \times x_1 + 3600 \times x_2 \geq 20000 $

**2. Machine Runtime Limit**  
$ x_1 \leq 18 $  
$ x_2 \leq 9 $



## Pyomo Implementation (QP)

The Pyomo modeling workflow remains the same as in the LP and MILP notebooks:

> define the model → variables → objective → constraints → solver → results

The key difference in this notebook is **how the objective function is defined**,  
since it now includes **quadratic terms**.

### Import Libraries

In [1]:
import pyomo.environ as pyomo
from pyomo.environ import value

### Create the Model

In [2]:
model = pyomo.ConcreteModel()

### Create Variables

In [3]:
# Decision Variables
## x1 and x2 would be in hour units therefore making the domain non negative reals so that we can have values like 1.1 hours
model.x1 = pyomo.Var(domain=pyomo.NonNegativeReals)
model.x2 = pyomo.Var(domain=pyomo.NonNegativeReals)

### Objective Function

In [4]:
def obj_func(model):
    return 0.4 * model.x1**2 - 5 * model.x1 + model.x2**2 - 6*model.x2 + 50

model.objective = pyomo.Objective(rule = obj_func,
                                  sense = pyomo.minimize
                                  )

### Constraints

In [5]:
model.c = pyomo.ConstraintList()

# produce at most 20000 units
model.c.add(expr= 2700 * model.x1 + 3600 * model.x2 >= 20000)

# machine runtime limit
model.c.add(expr= model.x1 <= 18)
model.c.add(expr= model.x2 <= 9)

### Solver

We use **HiGHS** to solve this Quadratic Programming (QP) problem, since HiGHS natively supports
**convex QP** and is optimized for this class of problems.

Alternatively, **IPOPT** can also be used. IPOPT is a general-purpose
**Nonlinear Programming (NLP)** solver and is not specialized for QP.
However, since a QP is a **special case of an NLP**, IPOPT is still able to solve it.

For small demonstration problems, IPOPT works perfectly well.
For larger or more complex QP problems, a dedicated QP solver such as HiGHS
is usually faster and more efficient.

For demonstration purposes, we compare IPOPT and HiGHS to highlight the
performance differences between a general NLP solver and a specialized QP solver.


#### IPOPT Solver

In [6]:
ipopt_solver = pyomo.SolverFactory("ipopt")

In [7]:
ipopt_results = ipopt_solver.solve(model)

In [8]:
print(ipopt_results)


Problem: 
- Lower bound: -.inf
  Upper bound: .inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.14.19\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.21256303787231445
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [9]:
print("ipopt solver results")
print(f"x1 = \t \t \t \t \t{value(model.x1)}")
print(f"x2 = \t \t \t \t \t{value(model.x2)}")
print(f"minimized cost = \t \t \t{value(model.objective)}")
print(f"Units produced =\t \t \t{2700 * value(model.x1) + 3600 * value(model.x2)}")
print(f"cost per unit produced (cost/unit) = \t{value(model.objective)/(2700 * value(model.x1) + 3600 * value(model.x2))}")

ipopt solver results
x1 = 	 	 	 	 	6.2500000013347705
x2 = 	 	 	 	 	3.000000000795621
minimized cost = 	 	 	25.375000000000004
Units produced =	 	 	27675.000006468115
cost per unit produced (cost/unit) = 	0.0009168925020440628


#### HiGHS Solver

In [10]:
from pyomo.contrib.solver.solvers.highs import Highs

# Define the solver
highs_solver = Highs()

In [11]:
highs_results = highs_solver.solve(model)

In [12]:
print(highs_results.solver_log)

QP has 3 rows; 2 cols; 4 matrix nonzeros; 2 Hessian nonzeros
Coefficient ranges:
  Matrix  [1e+00, 4e+03]
  Cost    [5e+00, 6e+00]
  Hessian [8e-01, 2e+00]
  Bound   [0e+00, 0e+00]
  RHS     [9e+00, 2e+04]
  Iteration        Objective     NullspaceDim
          0        89.600016                0      0.00s
          6        112.00001                1      0.00s
Model status        : Optimal
QP ASM    iterations: 6
Objective value     :  1.1199998791e+02
P-D objective error :  9.9624765435e-01
HiGHS run time      :          0.00



In [13]:
print("HiGHS solver results")
print(f"x1 = \t \t \t \t \t{value(model.x1)}")
print(f"x2 = \t \t \t \t \t{value(model.x2)}")
print(f"minimized cost = \t \t \t{value(model.objective)}")
print(f"Units produced =\t \t \t{2700 * value(model.x1) + 3600 * value(model.x2)}")
print(f"cost per unit produced (cost/unit) = \t{value(model.objective)/(2700 * value(model.x1) + 3600 * value(model.x2))}")

HiGHS solver results
x1 = 	 	 	 	 	17.499998656250163
x2 = 	 	 	 	 	9.0
minimized cost = 	 	 	111.99998790625219
Units produced =	 	 	79649.99637187544
cost per unit produced (cost/unit) = 	0.0014061518268417598


### Results Intepretation

#### IPOPT Solver
The IPOPT solver converges in 0.163 s according to the solver log and approximately 0.1 s based on notebook execution time. The obtained solution is: 
```
ipopt solver results
x1 = 	 	 	 	 	6.2500000013347705
x2 = 	 	 	 	 	3.000000000795621
minimized cost = 	 	 	25.375000000000004
Units produced =	 	 	27675.000006468115
cost per unit produced (cost/unit) = 	0.0009168925020440628
```
IPOPT produces only slightly above the required production level, resulting in the **lowest total cost** and **lowest cost per unit**.
#### HiGHS solver
The HiGHS solver reports a runtime of 0.00 s in the solver log; however, the notebook execution time is similar to IPOPT at approximately 0.1 s. The resulting solution is:
```
HiGHS solver results
x1 = 	 	 	 	 	17.499998656250163
x2 = 	 	 	 	 	9.0
minimized cost = 	 	 	111.99998790625219
Units produced =	 	 	79649.99637187544
cost per unit produced (cost/unit) = 	0.0014061518268417598
```
HiGHS operates both machines close to their upper limits, resulting in significantly higher production and total cost.

#### Conclusion
Interestingly, IPOPT and HiGHS return different optimal solutions. IPOPT produces close to the minimum required output, while HiGHS operates both machines near their upper limits. This difference arises because the production constraint is specified as a lower bound rather than an equality, **allowing overproduction without explicit penalty**. Since the objective **minimizes cost** only and <b style="color:red">does not include revenue or demand-matching incentives</b>, multiple feasible solutions exist. This highlights the importance of **properly modeling demand constraints** in optimization problems.